### Summary: This script will obtain additional columns of geo data for each property in the loaded real estate data set. 

WARNING: This script is expected to take approximately ~2 days to run given the approximately 300,000 rows of data. 

### Outline of Code: 
* Import Modules, Constants, and Functions.
* Load Raw Data
* Step 0: Initialize column changes for prebatch df
* Step 1: Initialize df_postBatch with first GeoCode Batch (Currently takes about 20 hours)
* (Steps 2-4) Attempt a few 'cleanup' loops. Diminishing returns after 3 or 4 cleanup loops.
    * Step 2: Filter only to rows that failed to match. 
    * Step 3: getTract_batchAddressGeocode only those that failed to match earlier. 
    * Step 4: Update postbatch with new data.
* Step 5: Save the aggregated postgeo file as a single csv (for inspection purposes only).
* Step 6: Check that list lengths and indexes of the original dataframe, prebatch dataframe, and postbatch dataframe are all the same.
* Step 7: Reverse column name changes and save as MarylandData_postGeo_orig2.csv. 
* Step 8: Load the previously saved "_orig2" file.
* Step 9: Identify rows with valid Lat/Long and loop until each row has an accompaning 'tract' value.
* Step 10: Save the final CSV and print the insane amount of time the script took to run. 

---

In [1]:
# Import Modules, Constants, and Functions.
%run ARV_Functions_Constants.ipynb
start_time = time.time()
file_path_census = r"intermediate_BatchCensusCode_files\\"

# Census Geocoder Documentation:
# https://pypi.org/project/censusgeocode/
# https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf

CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
# Load Raw Data
df = read_small_csvs_as_big_df("raw_real_estate_files//", na_keep = False, dtype_dict = {'Levels':'object'})


<timed exec>:50: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:50: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:50: DtypeWarning: Columns (25,47) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:50: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:50: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:50: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.


In [5]:
# Step 0: Initialize column changes for prebatch df
batchColumns_inputDict = {'FullStreetAddress':'Street address','City':'City','StateOrProvince':'State','PostalCode':'ZIP'}
batchColumns_inverseDict = {v: k for k, v in batchColumns_inputDict.items()}
outputBatchCols = ['address','match','tract','block']
df_preBatch = df.rename(columns = batchColumns_inputDict)
df_preBatch['Unique ID'] = df_preBatch.index

# Confirm no rows contain the dreaded '"'
if df_preBatch[df_preBatch["Street address"].str.contains('"',na=False)].empty:
    print("Correct! No Address has \" in it!")
else:
    print("Problem! Row has \" in it. Remove these rows! ")

Correct! No Address has " in it!


In [18]:
%%time
# Step 1: Initialize df_postBatch with first GeoCode Batch (Currently takes about 20 hours)
df_geo_merge1 = getTract_batchAddressGeocode(df_preBatch) 
df_postBatch = df_preBatch.join(df_geo_merge1[outputBatchCols], how='left', on='Unique ID',lsuffix='_duplicate1', rsuffix='_duplicate2')
print("\n",df_postBatch.match.value_counts(),sep="")

Starting datetime: 2022-07-09 15:44:53.579819
The raw data frame of 19980 rows is split into 2 dataframes of 9990 length each (9990 rows in the last df.)
Trying Batch: 1... 

In [21]:
%%time
# (Steps 2-4) Attempt a few 'cleanup' loops. Diminishing returns after 3 or 4 cleanup loops. 
print("df_postBatch.match.value_counts(): ",df_postBatch.match.value_counts(),"\n",sep="")
n=4
for i in range(n):
    if 'False' in df_postBatch.match.unique():
        # Step 2: Filter only to rows that failed to match. 
        df_preBatch_loop = df_postBatch.loc[df_postBatch.tract.isna(), :].copy(); print("Does the df_preBatch_loop index and Unique ID still match?:",(df_preBatch_loop['Unique ID'].values == df_preBatch_loop.index.values).all())

        # Step 3: getTract_batchAddressGeocode only those that failed to match earlier. 
        df_geo_merge_loop = getTract_batchAddressGeocode(df_preBatch_loop, cleanupLoop = True) 

        # Step 4: Update postbatch with new data.
        df_postBatch.update(df_geo_merge_loop.loc[:,outputBatchCols], join='left') # update() method directly changes calling object
        print(df_postBatch.match.value_counts(),"\n",sep="")


df_postBatch.match.value_counts(): True     313656
False     24148
Name: match, dtype: int64

Does the df_preBatch_loop index and Unique ID still match?: True
Starting datetime: 2021-03-05 20:21:44.829393
The raw data frame of 24148 rows is split into 3 dataframes of 9990 length each (4168 rows in the last df.)
Trying Batch: 1... SUCCESS. Batch: 1 correct.  	datetime: 2021-03-05 21:24:31.572172
Trying Batch: 2... SUCCESS. Batch: 2 correct.  	datetime: 2021-03-05 22:33:48.338373
Trying Batch: 3... SUCCESS. Batch: 3 correct.  	datetime: 2021-03-05 23:00:11.158025

Starting Merge... 	datetime: 2021-03-05 23:00:11.190014
len(_list_geo_dfs): 3
Broken batches:  dict_keys([]) 

Finished Merge.Returning df. 	datetime: 2021-03-05 23:00:11.237017
_df_geo_merge.match.value_counts(): False    21814
True      2334
Name: match, dtype: int64

True     315990
False     21814
Name: match, dtype: int64

Does the df_preBatch_loop index and Unique ID still match?: True
Starting datetime: 2021-03-05 23:00:

In [22]:
# Step 5: Save the aggregated postgeo file as a single csv (for inspection purposes only).
file_path_census_postgeo_merged_file = file_path_census + "\\postgeo_aggregated_total_file\postGeoBatch_Total_" + str(n)+ "_Loops.csv"  
df_postBatch.to_csv(file_path_census_postgeo_merged_file, index=False) 

In [23]:
# Step 6: Check that list lengths and indexes of the original dataframe, prebatch dataframe, and postbatch dataframe are all the same.
compareList = [df.index.values, df_preBatch.index.values, df_preBatch['Unique ID'].astype(int).values, 
               # df_geo_merge1.index.values, # Will only be the same on a total run. 
               df_postBatch.index.values, df_postBatch['Unique ID'].astype(int).values]  # loop dfs are filtered, so not included.
for indexList in compareList:
    print(indexList, len(indexList), "\n")
print([(ele == compareList[0]).all() for ele in compareList])
print("df_preBatch_loop.index.max() == df_geo_merge_loop.index.max() : ", df_preBatch_loop.index.max() == df_geo_merge_loop.index.max(), df_preBatch_loop.index.max(), df_geo_merge_loop.index.max() )

[     0      1      2 ... 337801 337802 337803] 337804 

[     0      1      2 ... 337801 337802 337803] 337804 

[     0      1      2 ... 337801 337802 337803] 337804 

[     0      1      2 ... 337801 337802 337803] 337804 

[     0      1      2 ... 337801 337802 337803] 337804 

[True, True, True, True, True]
df_preBatch_loop.index.max() == df_geo_merge_loop.index.max() :  True 337786 337786


In [24]:
# Step 7: Reverse column name changes and save as MarylandData_postGeo_orig2.csv. Set postBatch_output as the new df for inspection purposes.
df_postBatch_output = df_postBatch.rename(columns = batchColumns_inverseDict).copy()

print("Proportion of data successfully geocoded: ",round(df_postBatch_output.match.value_counts()[0]/(df_postBatch_output.match.value_counts()[0]+df_postBatch_output.match.value_counts()[1]),3))
print(df_postBatch_output.match.value_counts())
print(df_postBatch_output.shape)
df_postBatch_output.head(3)

# Save file as "..._orig2" and print the number of hours the entire script has taken up until this point. 
df_postBatch_output.to_csv(file_path_census + "MarylandData_postGeo_orig2.csv",index=False) 
print("--- %s hours ---" % round((time.time() - start_time)/3600,2))

Proportion of data successfully geocoded:  0.949
True     320570
False     17234
Name: match, dtype: int64
(337804, 87)


,FullStreetAddress,City,StateOrProvince,PostalCode,County,SchoolDistrictName,YearBuilt,YearBuiltEffective,ClosePrice,CloseDate,...,TotalGarageAndParkingSpaces,TotalPhotos,Latitude,Longitude,PublicRemarks,Unique ID,address,match,tract,block
0,11102 CROSS ROAD TRL,BRANDYWINE,MD,20613,PRINCE GEORGES,,1951,0,60000.0,2017-11-22,...,0,7,38.72173,-76.81473,"SOLD ""AS IS"". NO ACCESS TO THE HOUSE. LEVEL ...",0,"11102 CROSS ROAD TRL, BRANDYWINE, MD, 20613",True,801004,1005
1,4506 CEDELL PL,TEMPLE HILLS,MD,20748,PRINCE GEORGES,PRINCE GEORGE'S COUNTY PUBLIC SCHOOLS,1965,0,309000.0,2017-10-15,...,1,14,38.80514,-76.93090,Must See Home! 4 Bedroom 3 Full Bath Detached ...,1,"4506 CEDELL PL, TEMPLE HILLS, MD, 20748",True,801901,3005
2,12607 WHITEHOLM DR,UPPER MARLBORO,MD,20774,PRINCE GEORGES,PRINCE GEORGE'S COUNTY PUBLIC SCHOOLS,1973,0,245000.0,2018-08-24,...,1,14,38.89964,-76.78679,Cash or FHA 203K loans only. Water is not avai...,2,"12607 WHITEHOLM DR, UPPER MARLBORO, MD, 20774",True,800521,3003


--- 26.81 hours ---


In [26]:
# Step 8: Load the previously saved "_orig2" file.
df2 = pd.read_csv(file_path_census + "MarylandData_postGeo_orig2.csv", keep_default_na=False, dtype={'Levels':'object'})

C:\Users\realc\AppData\Local\Temp\ipykernel_19132\4092739351.py:2: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(file_path_census + "MarylandData_postGeo_orig3.csv", keep_default_na=False, dtype={'Levels':'object'})


In [4]:
%%time

# Step 9: Identify rows with valid Lat/Long and loop until each row has an accompaning 'tract' value.
count = 0
latlonMask = (df2[['Latitude', 'Longitude']].notna().all('columns')&(df2['Latitude']!=0.0)&(df2['Longitude']!=0.0))
tractMask = (df2['tract']=='')
fullMask = latlonMask & (tractMask)

print("Total remaining rows left to censusGeocode:",len(df2[fullMask]) )
while (len(df2[fullMask]) >= 0) and (count < 12):  # count<10
    df_geoTemp = df2[fullMask].copy()
    df_geoTemp = getTract_LatLong(df_geoTemp)
    df2[fullMask] = df_geoTemp

    count += 1
    latlonMask = (df2[['Latitude', 'Longitude']].notna().all('columns')&(df2['Latitude']!=0.0)&(df2['Longitude']!=0.0))
    tractMask = (df2['tract']=="")
    fullMask = latlonMask & (tractMask)
    print("lac and lon that are both not NANs:", len(df2[latlonMask]))
    print("Tracts that are NANs:", len(df2[tractMask]))
    print("End Loop:", count, "\n")



Total remaining rows left to censusGeocode: 5
Success 20 37-L RIDGE RD #L 39.00335544 -76.8726714
Success 30 17401 CENTRAL AVE 38.89939899 -76.70377014
Success 31 16-K RIDGE RD #K 38.99843485 -76.87813466
Success 42 11250 KETTERING PL 38.89386 -76.80909
Success 91 9702 TAM O SHANTER DR 38.75726 -76.8015
lac and lon that are both not NANs: 100
Tracts that are NANs: 0
End Loop: 1 

lac and lon that are both not NANs: 100
Tracts that are NANs: 0
End Loop: 2 

lac and lon that are both not NANs: 100
Tracts that are NANs: 0
End Loop: 3 

lac and lon that are both not NANs: 100
Tracts that are NANs: 0
End Loop: 4 

lac and lon that are both not NANs: 100
Tracts that are NANs: 0
End Loop: 5 

lac and lon that are both not NANs: 100
Tracts that are NANs: 0
End Loop: 6 

lac and lon that are both not NANs: 100
Tracts that are NANs: 0
End Loop: 7 

lac and lon that are both not NANs: 100
Tracts that are NANs: 0
End Loop: 8 

lac and lon that are both not NANs: 100
Tracts that are NANs: 0
End Loo

In [33]:
%%time
# Step 10: Save the final CSV and print the insane amount of time the script took to run. 
save_big_df_as_small_csvs(df2, csv_output_path = "geocoded_real_estate_files//", base_name = "MarylandData_postGeo_orig3", row_max = 40000)
print("--- %s hours ---" % round((time.time() - start_time)/3600,2))

--- 0.15 hours ---
CPU times: total: 12.9 s
Wall time: 13.7 s


# TRASH CODE GOES HERE